In [40]:
from sentence_transformers import SentenceTransformer, util
from sklearn.cluster import AgglomerativeClustering
import numpy as np

In [ ]:
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.dimensionality import BaseDimensionalityReduction
from sklearn.linear_model import LogisticRegression

In [203]:
import glob
import json
import os.path

In [44]:
from nltk.stem import WordNetLemmatizer

In [52]:
from nltk import pos_tag
from nltk.corpus import wordnet

In [45]:
lemmatizer = WordNetLemmatizer()

In [2]:
import pandas as pd

In [229]:
from sklearn.feature_extraction.text import CountVectorizer


In [6]:
not_stemmed_word = pd.read_csv("../text_processing/ped_not_stemmed_word_based_wiki_score.csv")
corpus = not_stemmed_word["word"].tolist()

In [ ]:
with open('../text_processing/data/wiki_l1_ped_link_pt.json', 'r') as fp:
    all_links = json.load(fp)

In [136]:
corpus = list(all_links.keys())

In [177]:
len(set(list(all_links.keys())))

737

In [137]:
len(corpus)

737

In [138]:
def get_sim(u,v):

    emb1 = embedder.encode(u)
    if isinstance(v, list):
        v = ' '.join(v)
    emb2 = embedder.encode(v)
    
    return float(util.cos_sim(emb1, emb2)[0][0])

In [139]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [140]:
lem_words = []
for el in corpus:
    w = lemmatizer.lemmatize(el, get_wordnet_pos(el))
    lem_words.append(w)
lem_words = list(set(lem_words))

In [141]:
sim_s = []
for el in lem_words:
    sim_s.append(get_sim(el, 'pedestrian'))

In [142]:
df = pd.DataFrame({'words':lem_words, 'score':sim_s})
df = df.sort_values(by=['score'], ascending=False)
df

,words,score
547,pedestrian,1.000000
132,pedestrianize,0.807013
625,pedestrianism,0.792180
413,pedestriantraffic,0.781348
262,pedestrian crossing,0.756751
...,...,...
54,prose,0.079213
494,fundraise,0.065307
338,auto-free zones,0.052380
259,shared use path,0.052221


In [143]:
df = df[df['score'] > df['score'].quantile(0.25)]
df

,words,score
547,pedestrian,1.000000
132,pedestrianize,0.807013
625,pedestrianism,0.792180
413,pedestriantraffic,0.781348
262,pedestrian crossing,0.756751
...,...,...
79,british english,0.194685
336,unbelted,0.194659
204,gateless,0.194155
123,auto,0.194069


In [144]:
corpus = lem_words

In [145]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')

In [146]:
"""
This is a simple application for sentence embeddings: clustering
Sentences are mapped to sentence embeddings and then agglomerative clustering with a threshold is applied.
"""

corpus_embeddings = embedder.encode(corpus)

# Normalize the embeddings to unit length
corpus_embeddings = corpus_embeddings /  np.linalg.norm(corpus_embeddings, axis=1, keepdims=True)

# Perform clustering
# clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=2) #, affinity='cosine', linkage='average', distance_threshold=0.4)

clustering_model = AgglomerativeClustering(n_clusters=None, metric='cosine', linkage='average', distance_threshold=0.75) #, affinity='cosine', linkage='average', distance_threshold=0.4)


clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_

clustered_sentences = {}
for sentence_id, cluster_id in enumerate(cluster_assignment):
    if cluster_id not in clustered_sentences:
        clustered_sentences[cluster_id] = []

    clustered_sentences[cluster_id].append(corpus[sentence_id])



In [147]:
for i, cluster in clustered_sentences.items():
    print("Cluster ", i+1)
    print(cluster)
    print("")

Cluster  12
['fact', 'way', 'legwork', 'man', 'story', 'right', 'link', 'prosaic']

Cluster  5
['treadle', 'trample', 'handicapped', 'walk frame', 'underfoot', 'routine', 'horse gait', 'duck foot', 'feetless', 'footstep', 'footbreadth', 'stairfoot', 'by foot', 'clubfoot', 'clawfooted', 'roller shoe', 'footfall', 'exercise', 'hindfoot', 'footwrap', 'stroll', 'knucklewalk', 'forwalk', 'walk around', 'run', 'footwalk', 'firewalk', 'jaywalker', 'intentional walk', 'catwalk', 'ramp', 'have leg', 'whistle walk', 'footboy', 'gait', 'big toe', 'put on shoe', 'runway', 'quadrupedally', "hit one's stride", 'flatfoot', 'escalator', 'walk beat', 'bumblefoot', 'park', 'five toe', 'racewalking', 'activity', 'little toe', 'walkman', 'walk', 'foot', 'stop walk', 'walkabout', 'footmeal', 'random walk', 'parking', 'movement', 'treader', 'wheelchair', 'stride', 'jaywalk', 'roller skates', 'footway', 'webfoot', 'wheelaround', 'stroller', 'by walk', 'tramper', 'your foot', 'footprint', 'foot soldier', 'wal

In [148]:
len(clustered_sentences.items())

45

In [149]:
len(corpus)

670

In [150]:
from sklearn.cluster import KMeans


In [151]:

"""
This is a simple application for sentence embeddings: clustering
Sentences are mapped to sentence embeddings and then k-mean clustering is applied.
"""

corpus_embeddings = embedder.encode(corpus)

# Perform kmean clustering
num_clusters = 30
clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_

clustered_sentences = [[] for i in range(num_clusters)]
for sentence_id, cluster_id in enumerate(cluster_assignment):
    print(f's-id: {sentence_id}, c-id: {cluster_id}')
    clustered_sentences[cluster_id].append(corpus[sentence_id])



s-id: 0, c-id: 18
s-id: 1, c-id: 15
s-id: 2, c-id: 12
s-id: 3, c-id: 3
s-id: 4, c-id: 19
s-id: 5, c-id: 29
s-id: 6, c-id: 19
s-id: 7, c-id: 28
s-id: 8, c-id: 17
s-id: 9, c-id: 26
s-id: 10, c-id: 26
s-id: 11, c-id: 16
s-id: 12, c-id: 11
s-id: 13, c-id: 1
s-id: 14, c-id: 3
s-id: 15, c-id: 24
s-id: 16, c-id: 26
s-id: 17, c-id: 24
s-id: 18, c-id: 18
s-id: 19, c-id: 6
s-id: 20, c-id: 22
s-id: 21, c-id: 18
s-id: 22, c-id: 15
s-id: 23, c-id: 25
s-id: 24, c-id: 15
s-id: 25, c-id: 9
s-id: 26, c-id: 18
s-id: 27, c-id: 6
s-id: 28, c-id: 3
s-id: 29, c-id: 2
s-id: 30, c-id: 20
s-id: 31, c-id: 9
s-id: 32, c-id: 17
s-id: 33, c-id: 15
s-id: 34, c-id: 6
s-id: 35, c-id: 21
s-id: 36, c-id: 20
s-id: 37, c-id: 15
s-id: 38, c-id: 15
s-id: 39, c-id: 5
s-id: 40, c-id: 8
s-id: 41, c-id: 0
s-id: 42, c-id: 26
s-id: 43, c-id: 19
s-id: 44, c-id: 27
s-id: 45, c-id: 27
s-id: 46, c-id: 23
s-id: 47, c-id: 0
s-id: 48, c-id: 18
s-id: 49, c-id: 19
s-id: 50, c-id: 5
s-id: 51, c-id: 29
s-id: 52, c-id: 6
s-id: 53, c-id: 24


In [152]:
len(corpus_embeddings)

670

In [153]:
type(corpus_embeddings)

numpy.ndarray

In [154]:
clustering_model.score(embedder.encode(["pedestrian"]))

-0.24702131748199463

In [155]:
len(clustering_model.cluster_centers_)

30

In [156]:
c_id = []
labels = []
avg_score = []

for i, cluster in enumerate(clustered_sentences):
    c_sim = 0
    for el in cluster:
        c_sim += get_sim(el, 'pedestrian')
    print(f"average sim cluster {i} to pedestrian is: {c_sim/len(cluster)}")
    
    print(cluster)
    c_id.append(i)
    labels.append(cluster)
    avg_score.append(c_sim/len(cluster))
    print("")

average sim cluster 0 to pedestrian is: 0.2472480939080318
['landscape', 'enjoy nature', 'solace', 'naturestrip', 'natural environment', 'earthbound', 'wind', 'refuge', 'environment', 'shelter', 'nature', 'mankind']

average sim cluster 1 to pedestrian is: 0.275070890211142
['underfoot', 'duck foot', 'feetless', 'footstep', 'footbreadth', 'stairfoot', 'by foot', 'clubfoot', 'clawfooted', 'roller shoe', 'footfall', 'hindfoot', 'footwrap', 'footboy', 'big toe', 'put on shoe', 'flatfoot', 'bumblefoot', 'five toe', 'little toe', 'foot', 'footmeal', 'footway', 'webfoot', 'your foot', 'foot soldier', 'ten toe', 'two foot', 'acre foot', 'leg', "on one's foot", 'footly', 'footsore', 'megafoot', 'footsie', 'cleave foot', 'foothalt', 'dry foot', 'on foot', 'flat foot', 'tube foot', 'footpath', 'clawfoot', 'footstrike', 'footpeg', 'splayfoot', 'footswitch', 'kilofoot', 'foothold', 'footer', 'footrail', 'midfoot']

average sim cluster 2 to pedestrian is: 0.2263984622603113
['routine', 'reverie', '

average sim cluster 26 to pedestrian is: 0.3498621828854084
['rightofway', 'trafficwarden', 'dual carriageway', 'freeway', 'interstate', 'roadway', 'trafficcop', 'concourse', 'pavementartist', 'permanentway', 'streetsign', 'pavement', 'ramp', 'road surface', 'curbless', 'runway', 'street', 'roundabout', 'trafficisland', 'roadhog', 'walkway', 'trafficlane', 'traffic', 'parking', 'sidewalk', 'frontage road', 'intersection', 'wheelaround', 'carriageway', 'boardwalk', 'road rage', 'roadsurface', 'road', 'alleyway', 'trafficcircle', 'pound pavement', 'asphalt', 'highway', 'the highway code', 'undergroundrailway']

average sim cluster 27 to pedestrian is: 0.28422428614326883
['stomper', 'staggerer', 'wayfarer', 'stamper', 'hardshoulder', 'waddler', 'stomp', 'totterer', 'trekker', 'treader', 'stumbler', 'stalker', 'shuffler', 'stroller', 'marcher', 'nightrider', 'parader', 'hiker', 'streetcorner', 'reeler', 'blindcurve', 'plodder', 'tripper', 'swaggerer', 'trudger', 'hobbler', 'strider', 'lim

In [159]:
labels[4]

['promenade',
 'pedestrianize',
 'thoroughfare',
 'pedestrianisation',
 'pedestriantraffic',
 'pedestrian',
 'pedestrianism']

In [164]:
for i, cls in enumerate(labels):
    if 'walk frame' in cls:
        print(cls)

['walk frame', 'stroll', 'knucklewalk', 'forwalk', 'footwalk', 'firewalk', 'jaywalker', 'intentional walk', 'catwalk', 'whistle walk', 'walk beat', 'racewalking', 'walkman', 'walk', 'stop walk', 'walkabout', 'random walk', 'movement', 'jaywalk', 'roller skates', 'by walk', 'walkable', 'walkathon', 'bushwalk', 'walk in park', 'walk cane', 'warwalking', 'baby walker', 'spacewalk', 'walker']


In [186]:
def find_closest_cluster(word, labels):
    sim_score = []
    for label in labels:
        max_sim = 0
        for w in label:
            sim = get_sim(w, word)
            if sim > max_sim:
                max_sim = sim
        sim_score.append(max_sim)
    max_value = max(sim_score)
    max_index = sim_score.index(max_value)
    return max_index

In [173]:
len(corpus)

670

In [188]:
key_cid = dict()
for el in all_links:
    search_keys_cls = []
    for i, cls in enumerate(labels):
        if el in cls:
            search_keys_cls.append(i)
            key_cid[el] = i

    if not search_keys_cls:
        # find closest cluster
        print(el)
        key_cid[el] = find_closest_cluster(el, labels)
        print(key_cid[el])
#         break

dogs
21
checkpoints
5
plazas
25
excursions
23
mornings
7
skateboards
13
connecting
29
ramps
26
alleys
5
levels
6
lives
21
corridors
5
wards
5
factors
6
roads
26
pattens
15
crowded
18
connections
29
walking
16
speeding
6
types
11
footpaths
1
facilities
19
zones
7
underpasses
14
crosswalks
8
fatalities
17
congested
2
collisions
5
crossings
8
vehicles
10
legs
1
escalators
28
asphalted
26
boots
6
powers
6
womens
11
roadways
26
chances
18
lines
11
overpasses
14
guides
29
stairs
22
childs
21
neighborhoods
19
nights
21
ways
18
entrances
29
amenities
19
landscaped
0
sidewalks
26
protected
3
rails
9
parks
19
forces
6
signals
6
areas
19
paths
29
rights
26
running
15
tours
23
persons
13
motorized
12
movements
16
activities
2
trips
23
citys
19
highways
26
buildings
3
intoxicated
25
miles
5
exercises
2
pathways
29
precincts
19
thoroughfares
4
streets
26
bridges
14
students
21
interchanges
5
spaces
3
feats
28
lanes
5
junctions
29
enclosed
3
fundraising
13
adults
2
yorks
19
deaths
21
purposes
3
shelt

In [189]:
key_cid

{'cross street': 8,
 'stop walk': 16,
 'pavement': 26,
 'boardwalk': 26,
 'dogs': 21,
 'midfoot': 1,
 'roller shoe': 1,
 'pattern': 6,
 'totterer': 27,
 'motor': 12,
 'checkpoints': 5,
 'merit': 3,
 'approach': 22,
 'rambler': 13,
 'pedestrian crossing': 8,
 'plazas': 25,
 'europe': 19,
 'tour': 23,
 'staggerer': 27,
 'arterial': 11,
 'excursions': 23,
 'interstate': 26,
 'tread': 15,
 'bicycle': 20,
 'your foot': 1,
 'scenicroute': 23,
 'mile': 5,
 'unmotorized': 2,
 'use your leg': 22,
 'blindcurve': 27,
 'runway': 26,
 'mornings': 7,
 'death': 21,
 'limited-access': 29,
 'footbreadth': 1,
 'skateboards': 13,
 'canoe': 10,
 'obstruction': 3,
 'kilofoot': 1,
 'five toe': 1,
 'vehicular': 10,
 'floating bridge': 14,
 'deambulatory': 2,
 'quadrupedally': 28,
 'step in': 22,
 'nightrider': 27,
 'soul': 21,
 'undergroundrailway': 26,
 'bicyclerack': 20,
 'connecting': 29,
 'tube foot': 1,
 'obesity': 2,
 'trip': 23,
 'parader': 27,
 'control': 6,
 'nondriver': 12,
 'downtown': 19,
 'tollc

In [190]:
values = {i for i in key_cid if key_cid[i]==26}
values

{'alleyway',
 'asphalted',
 'boardwalk',
 'carriageway',
 'concourse',
 'curbless',
 'dual carriageway',
 'freeway',
 'freeways',
 'frontage road',
 'highway',
 'highways',
 'intersection',
 'intersections',
 'interstate',
 'parking',
 'paved',
 'pavement',
 'pavementartist',
 'permanentway',
 'pound pavement',
 'ramp',
 'ramps',
 'rightofway',
 'rights',
 'road',
 'road rage',
 'road surface',
 'roadhog',
 'roads',
 'roadsurface',
 'roadway',
 'roadways',
 'roundabout',
 'runway',
 'sidewalk',
 'sidewalks',
 'street',
 'streets',
 'streetsign',
 'the highway code',
 'traffic',
 'trafficcircle',
 'trafficcop',
 'trafficisland',
 'trafficlane',
 'trafficwarden',
 'undergroundrailway',
 'walkway',
 'wheelaround'}

In [192]:
org_keys = []
for ids, cls in enumerate(labels):
    values = list({i for i in key_cid if key_cid[i]==ids})
    org_keys.append(values)

In [295]:
labels[7]

['south',
 'bed',
 'morning',
 'carson',
 'southbound',
 'eastbound',
 'zone',
 'side',
 'west',
 'auto-free zones',
 'east-west',
 'north-south',
 'northbound',
 'westbound']

In [296]:
org_keys[7]

['bed',
 'zones',
 'west',
 'side',
 'zone',
 'south',
 'westbound',
 'southbound',
 'east-west',
 'mornings',
 'northbound',
 'carson',
 'morning',
 'auto-free zones',
 'eastbound',
 'north-south']

In [307]:
def df2_df1(df1, df2):
    cond = df2['f_name'].isin(df1['f_name'])
    df2.drop(df2[cond].index, inplace = True)
    return df2

In [345]:
dict_of_files = dict()
dict_of_idx =  dict()
root_dir = '/home/xtest/text_processing/data/wiki_texts/ped_pt'
for idxx, keys in enumerate(org_keys):
#     print(idxx, keys)
#     dict_of_files[idx]= []
    f_dict = dict()
    for k in keys:
        f_names = [f.rsplit('/', 1)[1] for f in all_links[k]]
        f_names = [f'{root_dir}/{f}.txt' for f in f_names if os.path.isfile(f'{root_dir}/{f}.txt')]
        f_dict[k] = pd.DataFrame({'f_name': f_names})
    
    sorted_key = []
    for k in sorted(f_dict, key=lambda k: len(f_dict[k])):
#         print(k, len(f_dict[k]))
        sorted_key.append(k)
    
    for idx, k in enumerate(sorted_key):
#         print(sorted_key[idx:], k)
        for el in sorted_key[idx:]:
            if el == k:
                continue
#             print(f'we are going to calc: {el} - {k}')
            f_dict[el] = df2_df1(f_dict[k], f_dict[el])
           
    
    # create id for each catagory
    dict_of_idx[idxx] = dict()
    # create dict of dict
    dict_of_files[idxx] = dict()
    for i, el in enumerate(f_dict):
        dict_of_idx[idxx][i] = el
        dict_of_files[idxx][i] = list(f_dict[el]['f_name'].tolist())      
        
#     break

In [346]:
dict_of_idx[0]

{0: 'solace',
 1: 'sheltered',
 2: 'mankind',
 3: 'enjoy nature',
 4: 'environment',
 5: 'nature',
 6: 'natural environment',
 7: 'refuge',
 8: 'earthbound',
 9: 'landscaped',
 10: 'naturestrip'}

In [347]:
len(dict_of_files)

30

In [348]:
len(dict_of_files[0])

11

In [349]:
len(dict_of_files[0][10])

10

In [350]:
dict_of_files[0][10]

['/home/xtest/text_processing/data/wiki_texts/ped_pt/Road_verge.txt',
 '/home/xtest/text_processing/data/wiki_texts/ped_pt/Median_strip.txt',
 '/home/xtest/text_processing/data/wiki_texts/ped_pt/Talk%3APedestrian.txt',
 '/home/xtest/text_processing/data/wiki_texts/ped_pt/School_zone.txt',
 '/home/xtest/text_processing/data/wiki_texts/ped_pt/Bushy_Creek_Trail.txt',
 '/home/xtest/text_processing/data/wiki_texts/ped_pt/Road.txt',
 '/home/xtest/text_processing/data/wiki_texts/ped_pt/Brooker_Highway.txt',
 '/home/xtest/text_processing/data/wiki_texts/ped_pt/Talk%3ACarriageway.txt',
 '/home/xtest/text_processing/data/wiki_texts/ped_pt/2007_June_3.txt',
 '/home/xtest/text_processing/data/wiki_texts/ped_pt/Goodwood_Road,_Adelaide.txt']

In [356]:
edit_line = "Our editors will review what you’ve submitted and determine whether to revise the article"

In [439]:
cluster_data = dict()
for cluster in dict_of_files:
#     if cluster == 10:
    cluster_data[cluster] = dict()
    docs = []
    categories = []

    s = 0
    for idx in dict_of_files[cluster]:

        s += len(dict_of_files[cluster][idx])

        for f_path in dict_of_files[cluster][idx]:
            with open(f_path, "r") as rf:
                fl = rf.read().splitlines()
                fl = [l.replace(edit_line, " ") if edit_line in l else l for l in fl]
                fl = [l.strip() for l in fl  if l != '']
                nl = concatenate_chunks(fl, 5)
                docs.extend(nl)
                cat = [idx] * len(nl)
                categories.extend(cat)
    cluster_data[cluster]['data'] = docs
    cluster_data[cluster]['target'] = categories
#     break

In [440]:
s

1585

In [441]:
len(cluster_data[10]['data'])

4634

In [442]:
new_f = list(set(files[10]))
docs_p = []
for fn in new_f:
    with open(fn, "r") as rf:
        fl = rf.read().splitlines()
        fl = [l.replace(edit_line, " ") if edit_line in l else l for l in fl]
        fl = [l.strip() for l in fl  if l != '']
        nl = concatenate_chunks(fl, 5)
        docs_p.extend(nl)

In [443]:
len(docs_p)

4634

In [421]:
s = 0
for el in dict_of_files[10]:
    print(len(dict_of_files[10][el]))
    s += len(dict_of_files[10][el])

100
52
97
73
100
10
4
0
3
1
10
24


In [422]:
s

474

In [511]:
dict_of_idx[11]

{0: 'style',
 1: 'separation',
 2: 'attention',
 3: 'unobservant',
 4: 'fashion',
 5: 'unlighted',
 6: 'flow',
 7: 'type',
 8: 'lines',
 9: 'stream',
 10: 'sight',
 11: 'yellowline',
 12: 'blocks',
 13: 'bustling',
 14: 'view',
 15: 'egress',
 16: 'variety',
 17: 'emphasis',
 18: 'women',
 19: 'unbelted',
 20: 'slog',
 21: 'ft',
 22: 'prose',
 23: 'tree-lined',
 24: 'lameness',
 25: 'amble',
 26: 'womens',
 27: 'maindrag',
 28: 'arterial',
 29: 'types',
 30: 'carriage',
 31: 'block',
 32: 'circulation',
 33: 'woman',
 34: 'line',
 35: 'manner',
 36: 'verse',
 37: 'dress',
 38: 'end'}

In [451]:
list(set(list(dict_of_idx[10].values())))

['vehiculartraffic',
 'trolleycar',
 'seat belt',
 'canoe',
 'vehicular',
 'vehicle',
 'vehiculation',
 'vehicles',
 'vehicular traffic',
 'car',
 'travel by car',
 'automobile']

In [470]:
names = list(set(list(dict_of_idx[10].values())))
names

['vehiculartraffic',
 'trolleycar',
 'seat belt',
 'canoe',
 'vehicular',
 'vehicle',
 'vehiculation',
 'vehicles',
 'vehicular traffic',
 'car',
 'travel by car',
 'automobile']

In [449]:
len(cluster_data[10]['data'])

4634

In [450]:
len(cluster_data[10]['target'])

4634

In [471]:
classes = [names[i] for i in cluster_data[10]["target"]]

In [473]:
len(classes)

4634

In [474]:
classes[-20:]

['automobile',
 'automobile',
 'automobile',
 'automobile',
 'automobile',
 'automobile',
 'automobile',
 'automobile',
 'automobile',
 'automobile',
 'automobile',
 'automobile',
 'automobile',
 'automobile',
 'automobile',
 'automobile',
 'automobile',
 'automobile',
 'automobile',
 'automobile']

### Supervised

In [502]:
empty_dimensionality_model = BaseDimensionalityReduction()
clf = LogisticRegression()
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)
vectorizer_model = CountVectorizer(stop_words="english", ngram_range=(1, 3))

In [503]:
topic_model = BERTopic(vectorizer_model=vectorizer_model, diversity=0.8, umap_model=empty_dimensionality_model,
        hdbscan_model=clf,
        ctfidf_model=ctfidf_model)

In [504]:
topics, probs = topic_model.fit_transform(cluster_data[11]['data'], y=np.array(cluster_data[11]['target']))

In [505]:
topic_model.get_topic_info()

,Topic,Count,Name
0,0,1627,0_campus_american_population_park
1,1,1499,1_april 2006_bus_building_myanmar
2,2,929,2_tesla_parking_rumble_sms
3,3,868,3_metro_busch_anacostia_service
4,4,854,4_central park_crossings_frightened rabbit_pre...
5,5,803,5_lights_2014_motorway_game watch
6,6,796,6_truss_paris_signs_tactile
7,7,735,7_ballarat_huntington_istanbul_population
8,8,626,8_zebra_stop_vehicles_signals
9,9,612,9_talk_literature_harry potter_holmes


In [508]:
len(cluster_data[11]['data'])

17116

In [516]:
ped_similar_topics_a, ped_similarity_a = topic_model.find_topics("fashion", top_n=38)
print(ped_similar_topics_a)

[15, 11, 18, 35, 6, 24, 12, 7, 10, 23, 20, 3, 21, 9, 28, 1, 16, 5, 19, 36, 22, 8, 26, 31, 25, 27, 32, 0, 34, 13, 38, 29, 33, 2, 4, 37, 17, 14]


In [517]:
for t in ped_similar_topics_a:
    l = []
    print(t)
    for el in topic_model.get_topic(t):
        l.append(el[0])
    print(l)
#     print(ped_auto_model_a.get_topic(t))
    print('\n')

15
['fashion', 'le corbusier', 'sicilian', 'paris', 'lolita', 'xiv', 'unicorn', 'clements', 'shopping centers', 'bintang']


11
['thai', 'costumes', 'lenfant', 'dolly', 'security officers', 'new', 'san andreas', 'colonial williamsburg', 'square', 'onehunga']


18
['sustainable', 'urban design', 'plan', 'transport', 'psychology', 'rider', 'singapore', 'cpted', 'compact', 'emphasis']


35
['preddy', 'manni', 'ghost shoes', 'las vegas', 'skipping girl', 'billie', 'arash', 'korean', 'films', 'incident']


6
['truss', 'paris', 'signs', 'tactile', 'brooklyn bridge', 'roundabouts', 'used new south', 'designed', 'green', 'walking']


24
['houston', 'block', 'carfree', 'shopping', 'stone', 'zones', 'pedestrian streets', 'historic', 'olvera', 'descriptors']


12
['world trade center', 'trade', 'pennsylvania', 'road signs', 'inca', 'limousine', 'sight distance', 'excursion', 'impairment', 'design']


7
['ballarat', 'huntington', 'istanbul', 'population', 'docklands', 'century', 'building', 'inclu

In [486]:
hierarchical_topics = topic_model.hierarchical_topics(cluster_data[10]['data'])


100%|███████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 11.29it/s]


In [487]:
topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)


In [488]:
tree = topic_model.get_topic_tree(hierarchical_topics)
print(tree)

.
├─rights_toll roads_footpath_ashanti_layer
│    ├─■──s1 gard_right wheels_pts_bernoullis principle_object greater ── Topic: 10
│    └─rights way_england_toll roads_footpath_ashanti
│         ├─■──toll roads_layer_macadam_tar_scales ── Topic: 8
│         └─■──rights_england_ramblers_definitive_ashanti ── Topic: 9
└─used_new_safety_pedestrian_roads
     ├─used_new_bridge_speed_pedestrian
     │    ├─new_transit_station_streetcars_avenue
     │    │    ├─■──water_duluth_trails_built_tanana ── Topic: 2
     │    │    └─■──streetcars_bridge_light rail_new_tram ── Topic: 3
     │    └─used_safety_drivers_states_signs
     │         ├─used_safety_new_signs_car
     │         │    ├─■──used_pedestrian_sign_roads_limit ── Topic: 0
     │         │    └─safety_rear_transport_standard_lights
     │         │         ├─■──transport_prt_ring_cities_frontage roads ── Topic: 5
     │         │         └─■──safety_standard_lights_available_volvo ── Topic: 1
     │         └─■──suv_articles_july 2008

## Guided Topic Modeling

In [463]:
# get cluster seed
cluster_seed = list(set(list(dict_of_idx[10].values()) + ['pedestrian']))

In [464]:
vectorizer_model = CountVectorizer(stop_words="english", ngram_range=(1, 3))
topic_model = BERTopic(vectorizer_model=vectorizer_model, diversity=0.8, 
                       seed_topic_list=[cluster_seed], nr_topics='auto')
topics, probs = topic_model.fit_transform(cluster_data[10]['data'])

In [465]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,1424,-1_traffic_used_new_park
1,0,825,0_streetcar_new_used_route
2,1,213,1_left_roundabouts_signpost_curve
3,2,157,2_selfdriving_technology_crash_sae
4,3,101,3_trim_ford_features_model year
...,...,...,...
70,69,12,69_notificationstolen_unlockclimate controls_c...
71,70,11,70_aheadroad closed_3kmcontinues mcontinues_20...
72,71,11,71_terminal_gates_yvr_domestic
73,72,10,72_numberroute_map_guiden_lane


In [466]:
ped_similar_topics_a, ped_similarity_a = topic_model.find_topics("vehicle", top_n=10)
print(ped_similar_topics_a)

[14, 60, 58, 28, 32, 31, 3, 42, 50, 30]


In [467]:
for t in ped_similar_topics_a:
    l = []
    print(t)
    for el in topic_model.get_topic(t):
        l.append(el[0])
    print(l)
#     print(ped_auto_model_a.get_topic(t))
    print('\n')

14
['suvs', 'sold', 'singapore', 'unibody', 'cherokee', 'economy', 'fourwheel', 'compact', 'crossover', 'average']


60
['hybrid', 'warning', 'electric vehicles', 'pedestrians', 'combustion', 'prius', 'audible', 'cabin', 'blind', 'avas']


58
['vehicle road user', 'user category', 'road', 'specified', 'plate indicating vehicles', 'unexpected change', 'indicating place', 'indicating unexpected', 'indicating end', 'stopping parking']


28
['airbag', 'occupants', 'occupant', 'standard', 'deployment', 'impact', 'sensors', 'sodium', 'chrysler', 'autoliv']


32
['volvo cars', 'championship', 'swedish', 's60', 'geely', 'martikn', 'announced', 'world', 'manufacturer', 'safety']


31
['school buses', 'chassis', 'blue bird', 'student', 'introduced', 'fullsize', 'states', 'stop', 'diesel', 'wayne']


3
['trim', 'ford', 'features', 'model year', 'sclass', 'new', 'generation', 'suspension', 'xlt', 'offered']


42
['human', 'data', 'dummies', 'hybrid iii', 'good', 'crash test', 'safety', 'suvs', 'wi

# KeyBert

In [491]:
from keybert import KeyBERT

In [492]:
kw_model = KeyBERT()
keywords = kw_model.extract_keywords(cluster_data[10]['data'])

In [493]:
vocabulary = [k[0] for keyword in keywords for k in keyword]
vocabulary = list(set(vocabulary))

In [494]:
vectorizer_model = CountVectorizer(stop_words="english", ngram_range=(1, 3), vocabulary=vocabulary)


In [495]:
topic_model = BERTopic(vectorizer_model=vectorizer_model, diversity=0.8, 
                       nr_topics='auto')

In [496]:
topics, probs = topic_model.fit_transform(docs)


/home/xtest/.conda/envs/reai/lib/python3.9/site-packages/bertopic/vectorizers/_ctfidf.py:69: RuntimeWarning:

divide by zero encountered in divide

/home/xtest/.conda/envs/reai/lib/python3.9/site-packages/bertopic/vectorizers/_ctfidf.py:69: RuntimeWarning:

divide by zero encountered in divide



In [497]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,3891,-1_new_park_used_pedestrian
1,0,8096,0_road_new_used_station
2,1,115,1_displaystyle_equilibrium_paradox_tfrac
3,2,52,2_emissions_reduce_cooling_policies
4,3,46,3_velocity_acda_formula_stop
5,4,33,4_coordinates_folkestedetcoordinates_34_dutch
6,5,28,5_airbag_crash_occupants_safety
7,6,27,6_disabilities_access_election_devices
8,7,26,7_markerstate_highway_marker_state
9,8,22,8_audi_tdi_optional_20


In [499]:
ped_similar_topics_a, ped_similarity_a = topic_model.find_topics("vehicle", top_n=11)
print(ped_similar_topics_a)

[8, 5, -1, 14, 3, 0, 11, 2, 13, 6, 15]


In [500]:
ped_similar_topics_a.remove(-1)

In [501]:
for t in ped_similar_topics_a:
    l = []
    print(t)
    for el in topic_model.get_topic(t):
        l.append(el[0])
    print(l)
#     print(ped_auto_model_a.get_topic(t))
    print('\n')

8
['audi', 'tdi', 'optional', '20', 'new', 'torque', 'turbocharged', 'seats', 'transmission', 'manual']


5
['airbag', 'crash', 'occupants', 'safety', 'collisions', 'sensors', 'standard', 'volvo', 'states', 'designed']


14
['parking', 'priority', 'dangers', 'perpendicular', 'toll', 'variant', 'prohibition', 'end', 'deformations', 'pollute']


3
['velocity', 'acda', 'formula', 'stop', 'driver', 'curve', 'maximum', 'average', 'headlight', 'intersection']


0
['road', 'new', 'used', 'station', 'south', 'national', 'united', 'crossing', 'time', 'tunnel']


11
['asphalt', 'used', 'deposits', 'patent', 'production', 'temperature', 'canada', 'method', 'ancient', 'million']


2
['emissions', 'reduce', 'cooling', 'policies', 'costs', 'trees', '2021', 'eu', 'wind', 'technologies']


13
['chevron', 'bend', 'rightsharp', 'marker', 'postreflexion', 'yellowblack', 'roadschevron', 'way', 'danger', 'plate']


6
['disabilities', 'access', 'election', 'devices', 'united', 'students', 'guidelines', 'ada

In [317]:
unsorted_key = org_keys[7]
root_dir = '/home/xtest/text_processing/data/wiki_texts/ped_pt'
f_dict = dict()
for k in unsorted_key:
    f_names = [f.rsplit('/', 1)[1] for f in all_links[k]]
    f_names = [f'{root_dir}/{f}.txt' for f in f_names if os.path.isfile(f'{root_dir}/{f}.txt')]
    f_dict[k] = pd.DataFrame({'f_name': f_names})
    print(k, len(f_names))


bed 100
zones 100
west 100
side 100
zone 100
south 100
westbound 100
southbound 100
east-west 100
mornings 100
northbound 100
carson 100
morning 100
auto-free zones 100
eastbound 100
north-south 100


In [318]:
sorted_key = []
for k in sorted(f_dict, key=lambda k: len(f_dict[k])):
    print(k, len(f_dict[k]))
    sorted_key.append(k)

bed 100
zones 100
west 100
side 100
zone 100
south 100
westbound 100
southbound 100
east-west 100
mornings 100
northbound 100
carson 100
morning 100
auto-free zones 100
eastbound 100
north-south 100


In [319]:
for idx, k in enumerate(sorted_key):
    print(sorted_key[idx:], k)
    for el in sorted_key[idx:]:
        if el == k:
            continue
        print(f'we are going to calc: {el} - {k}')
        f_dict[el] = df2_df1(f_dict[k], f_dict[el])
#     break

['bed', 'zones', 'west', 'side', 'zone', 'south', 'westbound', 'southbound', 'east-west', 'mornings', 'northbound', 'carson', 'morning', 'auto-free zones', 'eastbound', 'north-south'] bed
we are going to calc: zones - bed
we are going to calc: west - bed
we are going to calc: side - bed
we are going to calc: zone - bed
we are going to calc: south - bed
we are going to calc: westbound - bed
we are going to calc: southbound - bed
we are going to calc: east-west - bed
we are going to calc: mornings - bed
we are going to calc: northbound - bed
we are going to calc: carson - bed
we are going to calc: morning - bed
we are going to calc: auto-free zones - bed
we are going to calc: eastbound - bed
we are going to calc: north-south - bed
['zones', 'west', 'side', 'zone', 'south', 'westbound', 'southbound', 'east-west', 'mornings', 'northbound', 'carson', 'morning', 'auto-free zones', 'eastbound', 'north-south'] zones
we are going to calc: west - zones
we are going to calc: side - zones
we are g

In [320]:
sorted_key = []
for k in sorted(f_dict, key=lambda k: len(f_dict[k])):
    print(k, len(f_dict[k]))
    sorted_key.append(k)

zone 13
morning 13
eastbound 29
northbound 37
auto-free zones 40
east-west 42
north-south 43
mornings 62
south 66
southbound 69
side 74
west 84
westbound 86
carson 97
zones 98
bed 100


In [321]:
f_dict['bed']['f_name'].tolist()

['/home/xtest/text_processing/data/wiki_texts/ped_pt/Arroyo_(creek).txt',
 '/home/xtest/text_processing/data/wiki_texts/ped_pt/Tyne_cyclist_and_pedestrian_tunnels.txt',
 '/home/xtest/text_processing/data/wiki_texts/ped_pt/Hostile_architecture.txt',
 '/home/xtest/text_processing/data/wiki_texts/ped_pt/Rumble_strip.txt',
 '/home/xtest/text_processing/data/wiki_texts/ped_pt/Culvert.txt',
 '/home/xtest/text_processing/data/wiki_texts/ped_pt/Ada_Anderson.txt',
 '/home/xtest/text_processing/data/wiki_texts/ped_pt/Glossary_of_road_transport_terms.txt',
 '/home/xtest/text_processing/data/wiki_texts/ped_pt/Road_surface_marking.txt',
 '/home/xtest/text_processing/data/wiki_texts/ped_pt/Airbag.txt',
 '/home/xtest/text_processing/data/wiki_texts/ped_pt/Bumper_(car).txt',
 '/home/xtest/text_processing/data/wiki_texts/ped_pt/Smartphone_zombie.txt',
 '/home/xtest/text_processing/data/wiki_texts/ped_pt/Fullpower_Technologies.txt',
 '/home/xtest/text_processing/data/wiki_texts/ped_pt/Volume_One:_UnIndi

In [195]:
df_1 = pd.DataFrame({'c_id': c_id, 'cluster':labels, 'json_key':org_keys , 'score':avg_score})
df_1 = df_1.sort_values(by=['score'], ascending=False)
df_1

,c_id,cluster,json_key,score
4,4,"[promenade, pedestrianize, thoroughfare, pedes...","[pedestrians, pedestriantraffic, pedestrianism...",0.708186
16,16,"[walk frame, stroll, knucklewalk, forwalk, foo...","[walkabout, walk cane, stroll, intentional wal...",0.407751
8,8,"[pelican crossing, cross, crossstreet, interfe...","[crosswalks, pelican crossing, crossing, cross...",0.400158
26,26,"[rightofway, trafficwarden, dual carriageway, ...","[road rage, sidewalks, roadsurface, trafficcop...",0.349862
9,9,"[travel, travel to destination, subway, bus, c...","[transportationsystem, bus-only, commuter trai...",0.338677
10,10,"[trolleycar, car, vehicular traffic, seat belt...","[vehiculartraffic, car, trolleycar, seat belt,...",0.332366
20,20,"[horse gait, bipedal, bicycle, trek, dave kuns...","[dave kunst, bimble, horse gait, bipedalism, t...",0.310068
17,17,"[handicapped, advocacy, fatality, accident, ch...","[plaintiff, injuries, accident, charitable org...",0.297370
12,12,"[motorist, driver, auto, passenger, carpooler,...","[drivearound, mobility scooter, nondriver, mot...",0.286285
27,27,"[stomper, staggerer, wayfarer, stamper, hardsh...","[plodder, hiker, stomp, stomper, staggerer, tr...",0.284224


In [199]:
len(org_keys)

30

In [ ]:

file_names = [f.rsplit('/', 1)[1] for f in f_names]
l_split, r_split = l.rsplit('/', 1)
if r_split == '':
    file_name = l_split.rsplit('/', 1)[1]
else:
    file_name = l.rsplit('/', 1)[1]
if file_name + '.txt' not in file_names:
    print(f"retrive {l, file_name}")
    retrive(l, file_name, dataset=dataset)

In [208]:
file_name = 'Museum_Campus'
os.path.isfile(f'/home/xtest/text_processing/data/wiki_texts/ped_pt/{file_name}.txt')

True

In [405]:
root_dir = '/home/xtest/text_processing/data/wiki_texts/ped_pt'
files = []
for cls in org_keys:
    f_names = []
    for fn in cls:
        f_names.extend(all_links[fn])
    f_names = [f.rsplit('/', 1)[1] for f in f_names]
    f_names = [f'{root_dir}/{f}.txt' for f in f_names if os.path.isfile(f'{root_dir}/{f}.txt')]
    files.append(list(set(f_names)))

In [406]:
files[10]

['/home/xtest/text_processing/data/wiki_texts/ped_pt/Braess%27s_paradox.txt',
 '/home/xtest/text_processing/data/wiki_texts/ped_pt/Trolleybus.txt',
 '/home/xtest/text_processing/data/wiki_texts/ped_pt/Quinebaug_River.txt',
 '/home/xtest/text_processing/data/wiki_texts/ped_pt/Footbridge.txt',
 '/home/xtest/text_processing/data/wiki_texts/ped_pt/Road_signs_in_Russia.txt',
 '/home/xtest/text_processing/data/wiki_texts/ped_pt/Segregated%20cycle%20facilities.txt',
 '/home/xtest/text_processing/data/wiki_texts/ped_pt/Defensive_driving.txt',
 '/home/xtest/text_processing/data/wiki_texts/ped_pt/Risk_compensation.txt',
 '/home/xtest/text_processing/data/wiki_texts/ped_pt/Personal_rapid_transit.txt',
 '/home/xtest/text_processing/data/wiki_texts/ped_pt/Car.txt',
 '/home/xtest/text_processing/data/wiki_texts/ped_pt/Epidemiology_of_motor_vehicle_collisions.txt',
 '/home/xtest/text_processing/data/wiki_texts/ped_pt/501_Queen.txt',
 '/home/xtest/text_processing/data/wiki_texts/ped_pt/List_of_countri

In [213]:
for f in files:
    print(len(f))

725
1580
1261
1986
410
1442
1598
953
437
1086
474
1948
864
1044
1066
1311
1465
572
1222
1823
584
1599
806
953
398
1115
1792
1471
553
1584


In [214]:
df_1 = pd.DataFrame({'c_id': c_id, 'cluster':labels, 'json_key':org_keys, 'files':files, 'score':avg_score})
df_1 = df_1.sort_values(by=['score'], ascending=False)
df_1

,c_id,cluster,json_key,files,score
4,4,"[promenade, pedestrianize, thoroughfare, pedes...","[pedestrians, pedestriantraffic, pedestrianism...",[/home/xtest/text_processing/data/wiki_tex...,0.708186
16,16,"[walk frame, stroll, knucklewalk, forwalk, foo...","[walkabout, walk cane, stroll, intentional wal...",[/home/xtest/text_processing/data/wiki_tex...,0.407751
8,8,"[pelican crossing, cross, crossstreet, interfe...","[crosswalks, pelican crossing, crossing, cross...",[/home/xtest/text_processing/data/wiki_tex...,0.400158
26,26,"[rightofway, trafficwarden, dual carriageway, ...","[road rage, sidewalks, roadsurface, trafficcop...",[/home/xtest/text_processing/data/wiki_tex...,0.349862
9,9,"[travel, travel to destination, subway, bus, c...","[transportationsystem, bus-only, commuter trai...",[/home/xtest/text_processing/data/wiki_tex...,0.338677
10,10,"[trolleycar, car, vehicular traffic, seat belt...","[vehiculartraffic, car, trolleycar, seat belt,...",[/home/xtest/text_processing/data/wiki_tex...,0.332366
20,20,"[horse gait, bipedal, bicycle, trek, dave kuns...","[dave kunst, bimble, horse gait, bipedalism, t...",[/home/xtest/text_processing/data/wiki_tex...,0.310068
17,17,"[handicapped, advocacy, fatality, accident, ch...","[plaintiff, injuries, accident, charitable org...",[/home/xtest/text_processing/data/wiki_tex...,0.297370
12,12,"[motorist, driver, auto, passenger, carpooler,...","[drivearound, mobility scooter, nondriver, mot...",[/home/xtest/text_processing/data/wiki_tex...,0.286285
27,27,"[stomper, staggerer, wayfarer, stamper, hardsh...","[plodder, hiker, stomp, stomper, staggerer, tr...",[/home/xtest/text_processing/data/wiki_tex...,0.284224


In [215]:
df_1.to_pickle(f'ped_cluster.pk')

In [216]:
def concatenate_chunks(string_list, chunk_size):
    if chunk_size > len(string_list):
        return [" ".join(string_list)]
    else:
        chunks = []
        for i in range(0, len(string_list), chunk_size):
            chunk = "".join(string_list[i:i + chunk_size])
            chunks.append(chunk)
        return chunks

In [218]:
edit_line = "Our editors will review what you’ve submitted and determine whether to revise the article"

In [418]:
len(new_f)

474

In [407]:
new_f = list(set(files[10]))
docs_p = []
for fn in new_f:
    with open(fn, "r") as rf:
        fl = rf.read().splitlines()
        fl = [l.replace(edit_line, " ") if edit_line in l else l for l in fl]
        fl = [l.strip() for l in fl  if l != '']
        nl = concatenate_chunks(fl, 5)
        docs_p.extend(nl)

In [423]:
len(docs_p)

4634

In [408]:
print([labels[10] ])

[['trolleycar', 'car', 'vehicular traffic', 'seat belt', 'vehicle', 'vehiculartraffic', 'vehicular', 'canoe', 'travel by car', 'vehiculation', 'automobile']]


In [409]:
vectorizer_model = CountVectorizer(stop_words="english", ngram_range=(1, 3))

In [410]:
topic_model = BERTopic(vectorizer_model=vectorizer_model, diversity=0.8, 
                       seed_topic_list=[['trolleycar', 'car', 'vehicular traffic', 'seat belt', 
                                         'vehicle', 'vehiculartraffic', 'vehicular', 'canoe', 
                                         'travel by car', 'vehiculation', 'automobile']], nr_topics='auto')

In [412]:
topics, probs = topic_model.fit_transform(docs_p)

In [413]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,1505,-1_used_new_safety_park
1,0,406,0_transit_avenue_new_station
2,1,132,1_left_yellow_stop_straight
3,2,128,2_ford_trim_available_xlt
4,3,94,3_weight_entry_prohibition_used
...,...,...,...
88,87,11,87_alaskan_ngi_potlatch_lower
89,88,11,88_zoning_private_dependency_american
90,89,10,89_path_compulsory track pedestrians_und_used ...
91,90,10,90_isa_vehicle_systems_beacons


In [452]:
ped_similar_topics_a, ped_similarity_a = topic_model.find_topics("vehicle", top_n=10)
print(ped_similar_topics_a)

[7, 1, 6, 10, 4, 5, 0, 3, 2, 8]


In [453]:
for t in ped_similar_topics_a:
    l = []
    print(t)
    for el in topic_model.get_topic(t):
        l.append(el[0])
    print(l)
#     print(ped_auto_model_a.get_topic(t))
    print('\n')

7
['suv', 'articles', 'july 2008', 'bridges', 'university', 'headlamps', 'unibody', 'infobox', 'selfdriving', 'fourwheel']


1
['safety', 'standard', 'lights', 'available', 'volvo', 'models', 'elevators', 'bus', 'expedition', 'states']


6
['acda', 'selfdriving', 'vehicles', 'safe speed', 'rule', 'rojas', 'clear distance', 'automation', 'assured', 'sight']


10
['s1 gard', 'right wheels', 'pts', 'bernoullis principle', 'object greater', 'pedestrians killed motor', 'involving rear right', 'wheels transit buses', 'curved', 'category']


4
['airbag', 'pedestrians', 'coverage', 'electric', 'automated', 'selfdriving', 'states', 'tram', 'warning', 'impact']


5
['transport', 'prt', 'ring', 'cities', 'frontage roads', 'hierarchy', 'active', 'emissions', 'cyclists', 'public']


0
['used', 'pedestrian', 'sign', 'roads', 'limit', 'right', 'red', 'safety', 'united', 'signals']


3
['streetcars', 'bridge', 'light rail', 'new', 'tram', 'ttc', 'tracks', 'san', 'began', 'systems']


2
['water', 'dulu

In [285]:
from sklearn.datasets import fetch_20newsgroups

data = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))
docs = data["data"]
categories = data["target"]
category_names = data["target_names"]

In [292]:
type(data["data"])

list

In [293]:
data["data"][0]

"\n\nI am sure some bashers of Pens fans are pretty confused about the lack\nof any kind of posts about the recent Pens massacre of the Devils. Actually,\nI am  bit puzzled too and a bit relieved. However, I am going to put an end\nto non-PIttsburghers' relief with a bit of praise for the Pens. Man, they\nare killing those Devils worse than I thought. Jagr just showed you why\nhe is much better than his regular season stats. He is also a lot\nfo fun to watch in the playoffs. Bowman should let JAgr have a lot of\nfun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final\nregular season game.          PENS RULE!!!\n\n"

In [294]:
category_names[0]

'alt.atheism'

In [287]:
len(categories)

18846

In [288]:
categories

array([10,  3, 17, ...,  3,  1,  7])

In [286]:
len(docs)

18846

In [283]:
topic_model = BERTopic(vectorizer_model=vectorizer_model, diversity=0.8
                       , nr_topics='auto')

In [284]:
topics, probs = topic_model.fit_transform(docs_p, y=labels[0])

ValueError: Length of x = 5917, length of y = 12, while it must be equal.